# Example: Zinc Powder from a Supplier

Let's describe an instance of some zinc powder with a set of properties defined in the specification sheet from the manufacturer!

This example covers a few topics:  

- How to describe a resource using ontology terms and JSON-LD  **[Basic]**
- How to run a simple query using SPARQL **[Moderate]**  
- How to use the ontology to fetch more information from other sources **[Advanced]**  

A live version of this notebook is available on Google Colab [here](https://colab.research.google.com/drive/19PxdZDPcKda8Ji6Nyzsz-_8KJFgNkmCa?usp=sharing)


## Describe the powder using ontology terms in JSON-LD format
The JSON-LD data that we will use is:

In [1]:
jsonld = {
  "@context": "https://w3id.org/emmo/domain/electrochemistry/context",
  "@type": ["Zinc", "Powder"],
  "schema:manufacturer": {
      "@id": "https://www.wikidata.org/wiki/Q680841",
      "schema:name": "Sigma-Aldrich"
  },
  "schema:productID": "324930",
  "schema:url": "https://www.sigmaaldrich.com/NO/en/product/aldrich/324930",
  "hasProperty": [
      {
        "@type": ["D95ParticleSize", "ConventionalProperty"],
        "hasNumericalPart": {
              "@type": "Real",
              "hasNumericalValue": 150
        },
        "hasMeasurementUnit": "emmo:MicroMetre",
        "dc:source": "https://www.sigmaaldrich.com/NO/en/product/aldrich/324930"
      }
  ]
}

## Parse this description into a graph
Now let's see how a machine would process this data by reading it into a Graph!

First, we install and import the python dependencies that we need for this example.

In [2]:
# Install and import dependencies
!pip install jsonschema rdflib requests matplotlib > /dev/null

import json
import rdflib
import requests
import sys
from IPython.display import Image, display
import matplotlib.pyplot as plt

We create the graph using a very handy python package called [rdflib](https://rdflib.readthedocs.io/en/stable/), which provides us a way to parse our json-ld data, run some queries using the language [SPARQL](https://en.wikipedia.org/wiki/SPARQL), and serialize the graph in any RDF compatible format (e.g. JSON-LD, Turtle, etc.).

In [3]:
# Create a new graph
g = rdflib.Graph()

# Parse our json-ld data into the graph
g.parse(data=json.dumps(jsonld), format="json-ld")

<Graph identifier=N9f0e3fae68794ed7b050c80a2dda9a81 (<class 'rdflib.graph.Graph'>)>

## Query the graph using SPARQL [Moderate]

Now, let's write a SPARQL query to get back some specific thing...like what is the name of the manufacturer?

In [4]:
query = """
PREFIX schema: <https://schema.org/>

SELECT ?manufacturerName
WHERE {
  ?thing schema:manufacturer ?manufacturer .
  ?manufacturer schema:name ?manufacturerName .
}
"""

# Execute the SPARQL query
results = g.query(query)

# Print the results
for row in results:
    print(row)


(rdflib.term.Literal('Sigma-Aldrich'),)


## Fetch additional information from other sources [Advanced]
Ontologies contain a lot of information about the meaning of things, but they don't always contain an exhaustive list of all the properties. Instead, they often point to other sources where that information exists rather than duplicating it. Let's see how you can use the ontology to fetch additional information from other sources.

First, we parse the ontology into the knowledge graph and retrieve the IRIs for the terms that we are interested in. In this case, we want to retrieve more information about Zinc from Wikidata, so we query the ontology to find Zinc's Wikidata ID.

In [5]:
# Parse the ontology into the knowledge graph
ontology = "https://raw.githubusercontent.com/emmo-repo/domain-electrochemistry/master/electrochemistry-inferred.ttl"
g.parse(ontology, format='turtle')

# Fetch the context
context_url = 'https://w3id.org/emmo/domain/electrochemistry/context'
response = requests.get(context_url)
context_data = response.json()

# Look for the IRI of Zinc in the context
zinc_iri = context_data.get('@context', {}).get('Zinc')
wikidata_iri = context_data.get('@context', {}).get('wikidataReference')

# Query the ontology to find the wikidata id for zinc
query = """
SELECT ?wikidataId
WHERE {
    <%s> <%s> ?wikidataId .
}
""" % (zinc_iri, wikidata_iri)

qres = g.query(query)
for row in qres:
    wikidata_id = row.wikidataId.split('/')[-1]

print(f"The Wikidata ID of Zinc: {wikidata_id}")

The Wikidata ID of Zinc: Q758


Now that we have the Wikidata ID for Zinc, we can query their SPARQL endpoint to retrieve some property. Let's ask it for the atomic mass.

In [6]:
# Query the Wikidata knowledge graph for more information about zinc
wikidata_endpoint = "https://query.wikidata.org/sparql"

# SPARQL query to get the atomic mass of zinc (Q758)
query = """
SELECT ?mass WHERE {
  wd:%s wdt:P2067 ?mass .
}
""" % wikidata_id

# Execute the request
response = requests.get(wikidata_endpoint, params={'query': query, 'format': 'json'})
data = response.json()

# Extract and print the mass value
mass = data['results']['bindings'][0]['mass']['value']
print(f"Wikidata says the atomic mass of zinc is: {mass}")

Wikidata says the atomic mass of zinc is: 65.38


We can also retrieve more complex data. For example, let's ask Wikidata to show us an image of zinc.

In [7]:
# SPARQL query to get the image of zinc (Q758)
query = """
SELECT ?image WHERE {
  wd:%s wdt:P18 ?image .
}
""" % wikidata_id

# Execute the request
response = requests.get(wikidata_endpoint, params={'query': query, 'format': 'json'})
data = response.json()

# Extract and display the image URL
if data['results']['bindings']:
    image_url = data['results']['bindings'][0]['image']['value']
    print(f"Image of Zinc: {image_url}")
    display(Image(url=image_url, width=300))  # Adjust width and height as needed

else:
    print("No image found for Zinc.")

Image of Zinc: http://commons.wikimedia.org/wiki/Special:FilePath/Zinc%20fragment%20sublimed%20and%201cm3%20cube.jpg
